# License Plate Detection Training - YOLOv5

Training model untuk deteksi plat nomor di Raspberry Pi 2 (edge device).

**Dataset**: Indonesian License Plate Dataset (~2000 images)

**Model**: YOLOv5 nano/small (optimized untuk edge device dengan resource terbatas)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_REPO/license-plate-detection-training.ipynb)

## 🚀 Google Colab Setup

In [ ]:
# Check if running on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Running on Google Colab")
    # Check GPU
    !nvidia-smi
else:
    print("✓ Running locally")

## 1. Setup Environment

In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -q -r requirements.txt
!pip install -q onnx onnxruntime kagglehub

In [ ]:
import os
import yaml
import shutil
from pathlib import Path
import glob
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import kagglehub

print("Environment setup complete!")

## 2. Download Dataset from Kaggle

In [ ]:
# Download dataset using kagglehub
print("Downloading Indonesian License Plate Dataset from Kaggle...")
path = kagglehub.dataset_download("juanthomaswijaya/indonesian-license-plate-dataset")

print(f"✓ Dataset downloaded to: {path}")
DATASET_ROOT = Path(path)

# List contents
print("\nDataset contents:")
for item in DATASET_ROOT.iterdir():
    print(f"  {item.name}")

# Find the correct DETECTION dataset folder
# The Kaggle dataset has 2 folders:
#   1. "Indonesian License Plate Dataset" - for DETECTION
#   2. "Indonesian License Plate Recognition Dataset" - for RECOGNITION (OCR)
detection_dataset_candidates = [
    DATASET_ROOT / 'Indonesian License Plate Dataset',
    DATASET_ROOT / 'indonesian-license-plate-dataset'
]

# Also search recursively in case of nested structure
detection_dataset_candidates.extend(list(DATASET_ROOT.glob('**/Indonesian License Plate Dataset')))

found = False
for candidate in detection_dataset_candidates:
    if candidate.exists() and (candidate / 'images').exists():
        DATASET_ROOT = candidate
        found = True
        print(f"\n✓ Found DETECTION dataset at: {DATASET_ROOT}")
        break

if not found:
    print("\n⚠️  DETECTION dataset folder not found automatically.")
    print("Available directories with 'images':")
    for item in Path(path).rglob('images'):
        if item.is_dir():
            print(f"  {item.parent} -> {item}")
    
    # Try to auto-select if only one images dir found
    images_dirs = list(Path(path).rglob('images'))
    if len(images_dirs) > 0:
        DATASET_ROOT = images_dirs[0].parent
        print(f"\n✓ Auto-selected: {DATASET_ROOT}")
    else:
        print("\n❌ Please manually set DATASET_ROOT in the next cell.")

## 3. Dataset Preparation & Exploration

In [ ]:
# Find images and labels directories
IMAGES_DIR = DATASET_ROOT / 'images'
LABELS_DIR = DATASET_ROOT / 'labels'

# Verify dataset structure
print(f"Dataset root exists: {DATASET_ROOT.exists()}")
print(f"Images dir exists: {IMAGES_DIR.exists()}")
print(f"Labels dir exists: {LABELS_DIR.exists()}")

# Count files
train_images = list((IMAGES_DIR / 'train').glob('*.jpg')) + list((IMAGES_DIR / 'train').glob('*.png'))
val_images = list((IMAGES_DIR / 'val').glob('*.jpg')) + list((IMAGES_DIR / 'val').glob('*.png'))
test_images = list((IMAGES_DIR / 'test').glob('*.jpg')) + list((IMAGES_DIR / 'test').glob('*.png'))

print(f"\n📊 Dataset Statistics:")
print(f"{'='*40}")
print(f"Train images: {len(train_images):>6}")
print(f"Val images:   {len(val_images):>6}")
print(f"Test images:  {len(test_images):>6}")
print(f"{'='*40}")
print(f"Total:        {len(train_images) + len(val_images) + len(test_images):>6}")

In [ ]:
# Visualize sample images with annotations
def visualize_sample(image_paths, label_dir, num_samples=4):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()
    
    samples = np.random.choice(image_paths, min(num_samples, len(image_paths)), replace=False)
    
    for idx, img_file in enumerate(samples):
        img = Image.open(img_file)
        img_w, img_h = img.size
        
        # Read corresponding label
        label_file = label_dir / (img_file.stem + '.txt')
        
        axes[idx].imshow(img)
        
        # Draw bounding boxes
        if label_file.exists():
            with open(label_file, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id, x_center, y_center, width, height = map(float, parts[:5])
                        
                        # Convert YOLO format to pixel coordinates
                        x_center *= img_w
                        y_center *= img_h
                        width *= img_w
                        height *= img_h
                        
                        x1 = x_center - width / 2
                        y1 = y_center - height / 2
                        
                        from matplotlib.patches import Rectangle
                        rect = Rectangle((x1, y1), width, height, linewidth=2, 
                                       edgecolor='red', facecolor='none')
                        axes[idx].add_patch(rect)
                        axes[idx].text(x1, y1-5, 'License Plate', color='red', 
                                     fontsize=10, weight='bold', 
                                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
        
        axes[idx].axis('off')
        axes[idx].set_title(f'Sample {idx+1}: {img_file.name}', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Visualize training samples
print("📸 Training Data Samples:")
visualize_sample(train_images, LABELS_DIR / 'train')

## 4. Create YOLOv5 Dataset Configuration

In [ ]:
# Create dataset.yaml for YOLOv5
dataset_yaml = {
    'path': str(DATASET_ROOT.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 1,  # number of classes
    'names': ['license_plate']
}

yaml_path = 'license_plate_detection.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f)

print(f"✓ Dataset config saved to: {yaml_path}")
print("\nContent:")
with open(yaml_path, 'r') as f:
    print(f.read())

## 5. Train YOLOv5 Nano (Optimized for Raspberry Pi)

In [ ]:
# Training configuration
MODEL = 'yolov5n'  # nano model (lightest, ~2MB)
# Alternative: yolov5s (small, ~7MB) jika butuh akurasi lebih tinggi

EPOCHS = 100
BATCH_SIZE = 16 if IN_COLAB else 8  # Larger batch for GPU
IMG_SIZE = 640
PROJECT = 'runs/train'
NAME = 'license_plate_detection'

print(f"🔧 Training Configuration:")
print(f"{'='*40}")
print(f"Model:       {MODEL}")
print(f"Epochs:      {EPOCHS}")
print(f"Batch size:  {BATCH_SIZE}")
print(f"Image size:  {IMG_SIZE}")
print(f"Device:      {'GPU (Colab)' if IN_COLAB else 'CPU/GPU'}")
print(f"{'='*40}")

In [ ]:
# Train the model
!python train.py \
    --img {IMG_SIZE} \
    --batch {BATCH_SIZE} \
    --epochs {EPOCHS} \
    --data {yaml_path} \
    --weights {MODEL}.pt \
    --project {PROJECT} \
    --name {NAME} \
    --cache \
    --patience 20 \
    --save-period 10

## 6. Comprehensive Model Evaluation

In [ ]:
# Load best model path
WEIGHTS_PATH = f'{PROJECT}/{NAME}/weights/best.pt'
print(f"📦 Loading best model from: {WEIGHTS_PATH}")

In [ ]:
# Validate on TEST set dengan detailed metrics
print("\n🧪 Testing on TEST SET...")
print("="*60)

!python val.py \
    --weights {WEIGHTS_PATH} \
    --data {yaml_path} \
    --img {IMG_SIZE} \
    --task test \
    --save-txt \
    --save-conf \
    --verbose

In [ ]:
# Parse validation results and calculate comprehensive metrics
import json

results_dir = Path(f'{PROJECT}/{NAME}')

# Read results from CSV if available
results_csv = results_dir / 'results.csv'
if results_csv.exists():
    df_results = pd.read_csv(results_csv)
    df_results.columns = df_results.columns.str.strip()
    
    print("\n📊 TRAINING METRICS SUMMARY")
    print("="*60)
    
    # Get final epoch metrics
    final_metrics = df_results.iloc[-1]
    
    # Print comprehensive metrics
    metrics_to_show = {
        'Precision': 'metrics/precision',
        'Recall': 'metrics/recall',
        'mAP@0.5': 'metrics/mAP_0.5',
        'mAP@0.5:0.95': 'metrics/mAP_0.5:0.95',
        'Train Loss (box)': 'train/box_loss',
        'Train Loss (obj)': 'train/obj_loss',
        'Val Loss (box)': 'val/box_loss',
        'Val Loss (obj)': 'val/obj_loss'
    }
    
    for metric_name, col_name in metrics_to_show.items():
        if col_name in final_metrics:
            value = final_metrics[col_name]
            print(f"{metric_name:.<30} {value:.4f}")
    
    print("="*60)
    
    # Calculate F1 Score
    if 'metrics/precision' in final_metrics and 'metrics/recall' in final_metrics:
        precision = final_metrics['metrics/precision']
        recall = final_metrics['metrics/recall']
        
        if precision + recall > 0:
            f1_score_val = 2 * (precision * recall) / (precision + recall)
            print(f"\n🎯 F1 SCORE: {f1_score_val:.4f}")
            print("="*60)
else:
    print("⚠️  Results CSV not found. Metrics will be shown from validation output.")

In [ ]:
# Visualize training results
from IPython.display import Image as IPImage, display

print("\n📈 TRAINING VISUALIZATIONS")
print("="*60)

# Display training curves
if (results_dir / 'results.png').exists():
    print("\n1️⃣  Training Curves:")
    display(IPImage(filename=str(results_dir / 'results.png')))

# Display confusion matrix
if (results_dir / 'confusion_matrix.png').exists():
    print("\n2️⃣  Confusion Matrix:")
    display(IPImage(filename=str(results_dir / 'confusion_matrix.png')))

# Display PR curve
if (results_dir / 'PR_curve.png').exists():
    print("\n3️⃣  Precision-Recall Curve:")
    display(IPImage(filename=str(results_dir / 'PR_curve.png')))

# Display F1 curve
if (results_dir / 'F1_curve.png').exists():
    print("\n4️⃣  F1 Confidence Curve:")
    display(IPImage(filename=str(results_dir / 'F1_curve.png')))

# Display validation predictions
if (results_dir / 'val_batch0_pred.jpg').exists():
    print("\n5️⃣  Validation Predictions (Batch 0):")
    display(IPImage(filename=str(results_dir / 'val_batch0_pred.jpg')))

## 7. Detailed Test Set Evaluation

In [ ]:
import torch
import time

# Load model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=WEIGHTS_PATH)
model.conf = 0.5  # Confidence threshold
model.iou = 0.45  # NMS IOU threshold

print("✓ Model loaded for inference")

In [ ]:
# Test on all test images and collect detailed metrics
print("\n🧪 Running inference on test set...")

test_results = {
    'true_positives': 0,
    'false_positives': 0,
    'false_negatives': 0,
    'inference_times': [],
    'confidences': []
}

for img_path in test_images[:50]:  # Test on subset for speed
    img = Image.open(img_path)
    
    # Ground truth
    label_file = LABELS_DIR / 'test' / (img_path.stem + '.txt')
    has_ground_truth = label_file.exists() and os.path.getsize(label_file) > 0
    
    # Inference
    start = time.time()
    results = model(img)
    inference_time = time.time() - start
    test_results['inference_times'].append(inference_time)
    
    # Count detections
    detections = results.pandas().xyxy[0]
    num_detections = len(detections)
    
    if num_detections > 0:
        test_results['confidences'].extend(detections['confidence'].tolist())
    
    # Simple TP/FP/FN calculation
    if has_ground_truth and num_detections > 0:
        test_results['true_positives'] += 1
    elif not has_ground_truth and num_detections > 0:
        test_results['false_positives'] += 1
    elif has_ground_truth and num_detections == 0:
        test_results['false_negatives'] += 1

print("✓ Inference complete!")

In [ ]:
# Calculate comprehensive test metrics
tp = test_results['true_positives']
fp = test_results['false_positives']
fn = test_results['false_negatives']

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

avg_inference = np.mean(test_results['inference_times']) * 1000  # ms
avg_conf = np.mean(test_results['confidences']) if test_results['confidences'] else 0

print("\n" + "="*60)
print("📊 COMPREHENSIVE TEST SET METRICS")
print("="*60)
print(f"\n🎯 Detection Performance:")
print(f"  True Positives:............ {tp}")
print(f"  False Positives:........... {fp}")
print(f"  False Negatives:........... {fn}")
print(f"\n📈 Classification Metrics:")
print(f"  Precision:................. {precision:.4f} ({precision*100:.2f}%)")
print(f"  Recall:.................... {recall:.4f} ({recall*100:.2f}%)")
print(f"  F1 Score:.................. {f1:.4f} ({f1*100:.2f}%)")
print(f"\n⚡ Performance Metrics:")
print(f"  Avg Inference Time:........ {avg_inference:.2f} ms")
print(f"  FPS:...................... {1000/avg_inference:.2f}")
print(f"  Avg Confidence:............ {avg_conf:.4f} ({avg_conf*100:.2f}%)")
print(f"\n📱 Raspberry Pi 2 Estimate:")
print(f"  Expected FPS:.............. 1-3 FPS")
print(f"  Expected Latency:.......... 500-1000 ms")
print("="*60)

## 8. Test Inference Speed Benchmark

In [ ]:
# Detailed speed benchmark
test_img = str(test_images[0])
img = Image.open(test_img)

# Warmup
print("Warming up model...")
for _ in range(10):
    _ = model(img)

# Benchmark
num_runs = 100
print(f"Running {num_runs} inference iterations...")
times = []
for _ in range(num_runs):
    start = time.time()
    results = model(img)
    times.append(time.time() - start)

times = np.array(times) * 1000  # Convert to ms

print("\n" + "="*60)
print("⚡ INFERENCE SPEED BENCHMARK")
print("="*60)
print(f"Runs:...................... {num_runs}")
print(f"Mean:...................... {times.mean():.2f} ms")
print(f"Median:.................... {np.median(times):.2f} ms")
print(f"Std Dev:................... {times.std():.2f} ms")
print(f"Min:....................... {times.min():.2f} ms")
print(f"Max:....................... {times.max():.2f} ms")
print(f"FPS (mean):................ {1000/times.mean():.2f}")
print("="*60)

# Plot distribution
plt.figure(figsize=(10, 5))
plt.hist(times, bins=30, color='skyblue', edgecolor='black')
plt.axvline(times.mean(), color='red', linestyle='--', label=f'Mean: {times.mean():.2f} ms')
plt.axvline(np.median(times), color='green', linestyle='--', label=f'Median: {np.median(times):.2f} ms')
plt.xlabel('Inference Time (ms)')
plt.ylabel('Frequency')
plt.title('Inference Time Distribution')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 9. Visual Test Results

In [ ]:
# Test detection on sample images
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.ravel()

sample_imgs = np.random.choice(test_images, min(9, len(test_images)), replace=False)

for idx, img_path in enumerate(sample_imgs):
    img = Image.open(img_path)
    results = model(img)
    
    # Render results
    rendered = results.render()[0]
    
    # Get detection info
    detections = results.pandas().xyxy[0]
    num_det = len(detections)
    avg_conf = detections['confidence'].mean() if num_det > 0 else 0
    
    axes[idx].imshow(rendered)
    axes[idx].axis('off')
    axes[idx].set_title(
        f'Detections: {num_det} | Conf: {avg_conf:.2f}',
        fontsize=10,
        color='green' if num_det > 0 else 'red'
    )

plt.tight_layout()
plt.suptitle('Test Set Detection Results', fontsize=16, y=1.01, weight='bold')
plt.show()

# Print detection details for first sample
if len(sample_imgs) > 0:
    print("\n📋 Sample Detection Details:")
    print("="*60)
    results = model(Image.open(sample_imgs[0]))
    print(results.pandas().xyxy[0].to_string())

## 10. Export Model for Deployment

In [ ]:
# Create export directory
export_dir = Path('../models/detection') if not IN_COLAB else Path('/content/models_export/detection')
export_dir.mkdir(parents=True, exist_ok=True)

print(f"✓ Export directory: {export_dir.absolute()}")

In [ ]:
# Export to ONNX (recommended for cross-platform)
print("Exporting to ONNX format...")
!python export.py \
    --weights {WEIGHTS_PATH} \
    --include onnx \
    --img {IMG_SIZE} \
    --simplify

# Copy exported model
onnx_path = Path(WEIGHTS_PATH).parent / 'best.onnx'
if onnx_path.exists():
    shutil.copy(onnx_path, export_dir / 'license_plate_detection.onnx')
    print(f"✓ ONNX model exported to: {export_dir / 'license_plate_detection.onnx'}")
    print(f"  Size: {os.path.getsize(export_dir / 'license_plate_detection.onnx') / 1024 / 1024:.2f} MB")

In [ ]:
# Also copy PyTorch weights
shutil.copy(WEIGHTS_PATH, export_dir / 'license_plate_detection.pt')
print(f"✓ PyTorch weights saved to: {export_dir / 'license_plate_detection.pt'}")
print(f"  Size: {os.path.getsize(export_dir / 'license_plate_detection.pt') / 1024 / 1024:.2f} MB")

In [ ]:
# Download models (for Colab)
if IN_COLAB:
    from google.colab import files
    
    print("\n📥 Download models:")
    print("Click to download ONNX model (for Raspberry Pi)")
    files.download(str(export_dir / 'license_plate_detection.onnx'))
    
    print("Click to download PyTorch model")
    files.download(str(export_dir / 'license_plate_detection.pt'))

## 11. Summary & Next Steps

In [ ]:
print("\n" + "="*60)
print("✅ LICENSE PLATE DETECTION TRAINING COMPLETE")
print("="*60)
print(f"\nModel: {MODEL}")
print(f"Training epochs: {EPOCHS}")
print(f"Best weights: {WEIGHTS_PATH}")
print(f"\n📦 Exported models:")
print(f"  ONNX:      {export_dir / 'license_plate_detection.onnx'}")
print(f"  PyTorch:   {export_dir / 'license_plate_detection.pt'}")
print(f"\n🎯 Final Metrics (Test Set):")
print(f"  F1 Score:  {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"\n⚡ Performance:")
print(f"  Inference: {avg_inference:.2f} ms")
print(f"  FPS:       {1000/avg_inference:.2f}")
print(f"\n📱 Next Steps:")
print(f"  1. Download ONNX model")
print(f"  2. Deploy to Raspberry Pi 2")
print(f"  3. Test real-time inference dengan camera")
print(f"  4. Train recognition model (next notebook)")
print("="*60)